**Step 1 Data Exploration And Loading**


In [14]:
#importing dependencies
import pandas as pd
import psycopg2
from sqlalchemy import create_engine


In [15]:
print(pd.__version__)

1.3.5


In [16]:
#CSV FILE JO HAI JIS P WORKING KARNI HAI WO READ KARWAO
db = pd.read_csv("Walmart.csv" , encoding_errors="ignore")


#IDENTIFY HOW MANY ROWS AND COLUMNS ARE THERE IN THAT CSV FILE
db.shape

(10051, 11)

In [17]:
#IT SHOWS TOP 5 VALUES OF THAT CSV FILE
db.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,$74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,$15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,$46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,$58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,$86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


In [18]:
#IT DESCRIBE THE CSV FILE BY SHOWING (COUNT , MEAN , STD , MIN_VALUE , MAX_VALUE)
db.describe()

,invoice_id,quantity,rating,profit_margin
count,10051.000000,10020.000000,10051.000000,10051.000000
mean,5025.741220,2.353493,5.825659,0.393791
std,2901.174372,1.602658,1.763991,0.090669
min,1.000000,1.000000,3.000000,0.180000
25%,2513.500000,1.000000,4.000000,0.330000
50%,5026.000000,2.000000,6.000000,0.330000
75%,7538.500000,3.000000,7.000000,0.480000
max,10000.000000,10.000000,10.000000,0.570000


In [19]:
#IT SHOWS THAT COLUMNS ALONG WITH THE COUNT OF ROWS AND THE DATA TYPE OF THAT COLUMN
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      10051 non-null  int64  
 1   Branch          10051 non-null  object 
 2   City            10051 non-null  object 
 3   category        10051 non-null  object 
 4   unit_price      10020 non-null  object 
 5   quantity        10020 non-null  float64
 6   date            10051 non-null  object 
 7   time            10051 non-null  object 
 8   payment_method  10051 non-null  object 
 9   rating          10051 non-null  float64
 10  profit_margin   10051 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 863.9+ KB


In [20]:
#IT SHOWS ALL THE DUPLICATE VALUES
db.duplicated().sum()

51

In [21]:
#THIS IS HOW YOU CAN REMOVE ALL THE DUPLICATE VALUES AND CHECK IF THE DUPLICATES VALUES ARE REMOVED OR NOT
db.drop_duplicates(inplace=True)
db.duplicated().sum()

0

In [22]:
#IT SHOWS ALL THE NULL VALUES IN THE COLUMN
db.isnull().sum()

invoice_id         0
Branch             0
City               0
category           0
unit_price        31
quantity          31
date               0
time               0
payment_method     0
rating             0
profit_margin      0
dtype: int64

In [23]:
#THIS IS HOW YOU CAN REMOVE ALL THE NULL VALUES AND CHECK IF THE NULL VALUES ARE REMOVED OR NOT
db.dropna(inplace=True)
db.isnull().sum()

invoice_id        0
Branch            0
City              0
category          0
unit_price        0
quantity          0
date              0
time              0
payment_method    0
rating            0
profit_margin     0
dtype: int64

In [24]:
#THIS IS HOW YOU CAN SEE THE DATA TYPES OF THE COLUMN
db.dtypes

invoice_id          int64
Branch             object
City               object
category           object
unit_price         object
quantity          float64
date               object
time               object
payment_method     object
rating            float64
profit_margin     float64
dtype: object

In [25]:
#THIS IS HOW YOU CAN CHANGE THE DATA TYPE OF ANY COLUMN BUT IN THIS CASE IT WILL GIVE AN ERROR BECAUSE OF A DOLLAR SIGN
db['unit_price'].astype(float)

ValueError: could not convert string to float: '$74.69'

In [26]:
#THIS IS HOW WE CAN CHANGE THE DATA TYPE AND CONVERT THE (OBJECT) INTO (FLOAT)
db['unit_price'] = db['unit_price'].str.replace('$', '').astype(float)

c:\Users\mudas\.pyenv\pyenv-win\versions\3.7.4\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [27]:
#CHECK IF THE $ IS REMOVED
db.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


In [28]:
#CHECK IF THE DATA TYPE IS CONVERTED TO FLOAT OR NOT
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9969 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      9969 non-null   int64  
 1   Branch          9969 non-null   object 
 2   City            9969 non-null   object 
 3   category        9969 non-null   object 
 4   unit_price      9969 non-null   float64
 5   quantity        9969 non-null   float64
 6   date            9969 non-null   object 
 7   time            9969 non-null   object 
 8   payment_method  9969 non-null   object 
 9   rating          9969 non-null   float64
 10  profit_margin   9969 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 934.6+ KB


In [29]:
#THIS IS HOW YOU CAN CHECK THE NAMES OF THE COLUMNS
db.columns

Index(['invoice_id', 'Branch', 'City', 'category', 'unit_price', 'quantity',
       'date', 'time', 'payment_method', 'rating', 'profit_margin'],
      dtype='object')

In [30]:
#THIS IS HOW YOU CAN ADD A NEW COLUMN (WITH CALCULATIONS)
db['total'] = db['unit_price'] * db['quantity']
db.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48,604.17


In [ ]:
#HERE THE DATA CLEANING THROUGH PANDAS AND JUPYTER NOTEBOOK IS DONE NOW WE HAVE TO CREATE THE SQL ENVIRONMENT

#1-OPEN A NEW TERMIAL
#2-PIP INSTALL PSYCOPG2 (FOR POSTGRE SQL)
#3-PIP INSTALL SQLALCHEMY
#4-IMPORT PSYCOPG2 AND ALCHEMY IN THE FIRST CODE WHERE WE IMPORTED (IMPORT PANDAS AS PD)

In [ ]:
#NOW WE HAVE TO MAKE A CONNECTION FROM HERE TO OUR SQL (POSTGRESQL) TO IMPORT THE DATA WE JUST CLEANED

# FOR POSTGRESQL

# host = localhost
# port = 5432
# user = postgres
# password = postgresql123

In [31]:
db.shape

(9969, 12)

In [32]:
#SAVE THE CLEAN DATA IN A CSV FORMAT
db.to_csv('Walmart_clean_data.csv', index=False)